In [1]:
# imports
import pandas as pd
from pandas import json_normalize
import sqlite3 as sq
import os
import requests
import json

In [2]:
#Key initialization
#set the keys
foursquare_id = os.environ["FOURSQUARE_API_KEY"]
foursquare_secret = os.environ["FOURSQUARE_SECRET"]

In [3]:
#set base url
url = "https://api.foursquare.com/v3/places/search?ll=43.656112128112106%2C-79.3802357750855&radius=1000&categories=10039&fields=rating%2Cname%2Cdistance%2Clocation"

#set header variable
headers = {"accept": "application/json",
           "authorization": foursquare_id}

In [4]:
#---Static-Variables-----

location = '43.656112128112106,-79.3802357750855'    #Young-Dundas Square
location2 = '43.725543525284145,-79.45215847331576'  #Yorkdale Mall
radius = 1000   #in meters


In [5]:
#Request function

def get_venues(urlstr, head):
    """
    Requests information from the parameter url
    Responds with status code
    if the code is not 204, return the response.
    """
    response = requests.get(urlstr, headers=head)
    response.raise_for_status()
    if response.status_code != 204:
        return response

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [6]:
data = get_venues(url, headers)

Parse through the response to get the POI details you want (rating, name, location, etc)

In [7]:
#parse data for conversion using .json() method
parsed_data = data.json()

Put your parsed results into a DataFrame

In [8]:
#Convert JSON to DataFrame using json_normalize()
df = pd.json_normalize(parsed_data['results'])

In [9]:
print('Music Venues within 1km of Young-Dundas Square (in meters):')

#sorting method to isolate columns needed, and make 
#bulky column names more legible with .rename()
sort_fs_df = df[['name', 'location.formatted_address', 'rating']].sort_values('rating', ascending=False).rename(columns={'location.formatted_address': 'address'}).fillna(0.0)

#convert the 10-point rating system to a 5-point rating
sort_fs_df['rating'] = sort_fs_df['rating']*0.5

#renumber the index after sorting by rating
fs_df = sort_fs_df.reset_index(drop=True)
fs_df

Music Venues within 1km of Young-Dundas Square (in meters):


,name,address,rating
0,Elgin & Winter Garden Theatre Ctr,"189 Yonge St (btwn Queen St E & Shuter St), To...",4.50
1,The Four Seasons Centre for the Performing Arts,"145 Queen St W (at University Ave.), Toronto O...",4.45
2,Jazz Bistro,251 Victoria St (btwn Dundas St. E & Shuter St...,4.25
3,Canadian Opera Co,"145 Queen St W (University), Toronto ON M5H 4G1",4.00
4,Betty Oliphant Theatre,"400 Jarvis St, Toronto ON M4Y 2G6",3.95
5,Buddies in Bad Times Theatre,"12 Alexander St (Yonge St), Toronto ON M4Y 1B4",3.90
6,Bar Plus,"2Nd Floor, Toronto ON M5B 1S5",3.10
7,Bluma Appel Theatre,"27 Front St E, Toronto ON M5E 1B4",3.10
8,Ryerson Theatre,"43 Gerrard St E (btwn Yonge & Church), Toronto...",3.00
9,Fusion Rock Radio,"10 Dundas St E (at Yonge St.), Toronto ON M5B 2G9",0.00


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [10]:
#yelp api initialize
yelp_api = os.environ["YELP_API_KEY"]
yelp_headers = {"accept": "application/json",
           "Authorization": f"BEARER {yelp_api}"}

#yelp query Variables
yelp_url = "https://api.yelp.com/v3/businesses/search?latitude=43.656112128112106&longitude=-79.3802357750855&radius=1000&categories=musicvenues&="

In [11]:
#yelp data pull
yelp_data = get_venues(yelp_url, yelp_headers)

Parse through your result and get POI details

In [12]:
yelp_parsed = yelp_data.json()

Put your parsed results into a DataFrame

In [13]:
df = pd.json_normalize(yelp_parsed['businesses'])

#sorting method to isolate columns needed, and make 
#bulky column names more legible with .rename()
sort_yp_df = df[['name', 'location.display_address', 'rating']].sort_values('rating', ascending=False).rename(columns={'location.display_address': 'address'})

#Renumber the index after sorting by rating 
yp_df = sort_yp_df.reset_index(drop=True)


In [14]:
#convert list values in address column to string, so it can be sent to sql
yp_df['address'] = yp_df['address'].apply(lambda x: str(x))

In [15]:
print('Music Venues within 1km of Young-Dundas Square (by rating):')
yp_df['address']

Music Venues within 1km of Young-Dundas Square (by rating):


0     ['54 The Esplanade', 'Toronto, ON M5E 1A6', 'C...
1     ['178 Victoria Street', 'Toronto, ON M5B 1T7',...
2     ['52  Wellington Street E', 'Toronto, ON M5E 1...
3     ['43 Gerrard Street E', 'Toronto, ON M5G 2A7',...
4     ['76 Church Street', 'Toronto, ON M5C 2G1', 'C...
5     ['First Canadian Place', '100 King St W', 'Tor...
6     ['67 Front Street E', 'Toronto, ON M5E 1B5', '...
7     ['251 Victoria Street', 'Toronto, ON M5B 1T8',...
8     ['250 Adelaide Street W', 'Toronto, ON M5H 1X6...
9     ['49 Front Street E', 'Toronto, ON M5E 1B3', '...
10    ['410 Sherbourne St', 'Toronto, ON M4X 1K2', '...
11    ['340 Yonge St', 'Toronto, ON M5B 1R8', 'Canada']
Name: address, dtype: object

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [16]:
# #create database
conn = sq.connect('../data/eda_data')


In [17]:
# #send dataframes to sql
yp_df.to_sql('locations', conn, if_exists='replace', index=False)

12

In [18]:
fs_df.to_sql('locations', conn, if_exists='append', index=False)

10

Get the top 10 venues according to their rating

In [19]:
result = pd.read_sql('SELECT rating, name\
                      FROM locations\
                      ORDER BY rating DESC\
                      LIMIT 10', conn)
print(result)

   rating                                             name
0    5.00                                    Bar Cathedral
1    4.50                Elgin & Winter Garden Theatre Ctr
2    4.45  The Four Seasons Centre for the Performing Arts
3    4.25                                      Jazz Bistro
4    4.00                                      Massey Hall
5    4.00                             The Reservoir Lounge
6    4.00                                  Ryerson Theatre
7    4.00                                   The Cavern Bar
8    4.00                                  Waterfall Stage
9    4.00                                Canadian Opera Co
